In [1]:
import os
os.environ['OMP_NUM_THREADS'] = '4'
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

Setting hyperparameters for llama2

In [2]:
from dataclasses import dataclass
@dataclass
class SerializerSettings:
    """
    Settings for serialization of numbers.

    Attributes:
    - base (int): The base for number representation.
    - prec (int): The precision after the 'decimal' point in the base representation.
    - signed (bool): If True, allows negative numbers. Default is False.
    - fixed_length (bool): If True, ensures fixed length of serialized string. Default is False.
    - max_val (float): Maximum absolute value of number for serialization.
    - time_sep (str): Separator for different time steps.
    - bit_sep (str): Separator for individual digits.
    - plus_sign (str): String representation for positive sign.
    - minus_sign (str): String representation for negative sign.
    - half_bin_correction (bool): If True, applies half bin correction during deserialization. Default is True.
    - decimal_point (str): String representation for the decimal point.
    """
    base: int = 10
    prec: int = 3
    signed: bool = True
    fixed_length: bool = False
    max_val: float = 1e7
    time_sep: str = ' ,'
    bit_sep: str = ' '
    plus_sign: str = ''
    minus_sign: str = ' -'
    half_bin_correction: bool = True
    decimal_point: str = ''
    missing_str: str = ' Nan'

In [3]:
llama2_hypers = dict(
    temp=0.7,
    alpha=0.95,
    beta=0.3,
    basic=False,
    settings=SerializerSettings(base=10, prec=3, signed=True, half_bin_correction=True)
)

Getting the CO2 MonaLua Dataset

In [13]:
from sklearn.datasets import fetch_openml

co2 = fetch_openml(data_id=41187, as_frame=True, parser='auto')
co2_data = co2.frame
co2_data["date"] = pd.to_datetime(co2_data[["year", "month", "day"]])
co2_data = co2_data.sort_values(by="date")
co2_data = co2_data[["date", "co2"]].set_index("date")

co2_data=co2_data.squeeze()
train, test = co2_data[:int(0.7*len(co2_data))], co2_data[int(0.7*len(co2_data)):]
print(train.shape,test.shape,co2_data.shape)

(1557,) (668,) (2225,)


In [9]:
from collections.abc import Iterable
import itertools, functools
import operator

class NoGetItLambdaDict(dict):
    """ Regular dict, but refuses to __getitem__ pretending
        the element is not there and throws a KeyError
        if the value is a non string iterable or a lambda """
    def __init__(self,d={}):
        super().__init__()
        for k,v in d.items():
            if isinstance(v,dict):
                self[k] = NoGetItLambdaDict(v)
            else:
                self[k] = v
    def __getitem__(self, key):
        value = super().__getitem__(key)
        if callable(value) and value.__name__ == "<lambda>":
            raise LookupError("You shouldn't try to retrieve lambda {} from this dict".format(value))
        if isinstance(value,Iterable) and not isinstance(value,(str,bytes,dict,tuple)):
            raise LookupError("You shouldn't try to retrieve iterable {} from this dict".format(value))
        return value
        
    # pop = __readonly__
    # popitem = __readonly__

def flatten(d, parent_key='', sep='/'):
    """An invertible dictionary flattening operation that does not clobber objs"""
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, dict) and v: # non-empty dict
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

def unflatten(d,sep='/'):
    """Take a dictionary with keys {'k1/k2/k3':v} to {'k1':{'k2':{'k3':v}}}
        as outputted by flatten """
    out_dict={}
    for k,v in d.items():
        if isinstance(k,str):
            keys = k.split(sep)
            dict_to_modify = out_dict
            for partial_key in keys[:-1]:
                try: dict_to_modify = dict_to_modify[partial_key]
                except KeyError:
                    dict_to_modify[partial_key] = {}
                    dict_to_modify = dict_to_modify[partial_key]
                # Base level reached
            if keys[-1] in dict_to_modify:
                dict_to_modify[keys[-1]].update(v)
            else:
                dict_to_modify[keys[-1]] = v
        else: out_dict[k]=v
    return out_dict

from collections import defaultdict
def sample_config(config_spec):
    """ Generates configs from the config spec.
        It will apply lambdas that depend on the config and sample from any
        iterables, make sure that no elements in the generated config are meant to 
        be iterable or lambdas, strings are allowed."""
    cfg_all = config_spec
    more_work=True
    i=0
    while more_work:
        cfg_all, more_work = _sample_config(cfg_all,NoGetItLambdaDict(cfg_all))
        i+=1
        if i>10: 
            raise RecursionError("config dependency unresolvable with {}".format(cfg_all))
    out = defaultdict(dict)
    out.update(cfg_all)
    return out

def _sample_config(config_spec,cfg_all):
    cfg = {}
    more_work = False
    for k,v in config_spec.items():
        if isinstance(v,dict):
            new_dict,extra_work = _sample_config(v,cfg_all)
            cfg[k] = new_dict
            more_work |= extra_work
        elif isinstance(v,Iterable) and not isinstance(v,(str,bytes,dict,tuple)):
            cfg[k] = random.choice(v)
        elif callable(v) and v.__name__ == "<lambda>":
            try:cfg[k] = v(cfg_all)
            except (KeyError, LookupError,Exception):
                cfg[k] = v # is used isntead of the variable it returns
                more_work = True
        else: cfg[k] = v
    return cfg, more_work

import random
class FixedNumpySeed(object):
    def __init__(self, seed):
        self.seed = seed
    def __enter__(self):
        self.np_rng_state = np.random.get_state()
        np.random.seed(self.seed)
        self.rand_rng_state = random.getstate()
        random.seed(self.seed)
    def __exit__(self, *args):
        np.random.set_state(self.np_rng_state)
        random.setstate(self.rand_rng_state)

class grid_iter(object):
    """ Defines a length which corresponds to one full pass through the grid
        defined by grid variables in config_spec, but the iterator will continue iterating
        past that by repeating over the grid variables"""
    def __init__(self,config_spec,num_elements=-1,shuffle=True):
        self.cfg_flat = flatten(config_spec)
        is_grid_iterable = lambda v: (isinstance(v,Iterable) and not isinstance(v,(str,bytes,dict,tuple)))
        iterables = sorted({k:v for k,v in self.cfg_flat.items() if is_grid_iterable(v)}.items())
        if iterables: self.iter_keys,self.iter_vals = zip(*iterables)
        else: self.iter_keys,self.iter_vals = [],[[]]
        self.vals = list(itertools.product(*self.iter_vals))
        if shuffle:
            with FixedNumpySeed(0): random.shuffle(self.vals)
        self.num_elements = num_elements if num_elements>=0 else (-1*num_elements)*len(self)

    def __iter__(self):
        self.i=0
        self.vals_iter = iter(self.vals)
        return self
    def __next__(self):
        self.i+=1
        if self.i > self.num_elements: raise StopIteration
        if not self.vals: v = []
        else:
            try: v = next(self.vals_iter)
            except StopIteration:
                self.vals_iter = iter(self.vals)
                v = next(self.vals_iter)
        chosen_iter_params = dict(zip(self.iter_keys,v))
        self.cfg_flat.update(chosen_iter_params)
        return sample_config(unflatten(self.cfg_flat))
    def __len__(self):
        product = functools.partial(functools.reduce, operator.mul)
        return product(len(v) for v in self.iter_vals) if self.vals else 1

In [11]:
out = {}
hypers = list(grid_iter({'model': 'llama-7b', **llama2_hypers}))
num_samples = 10
hypers

[defaultdict(dict,
             {'model': 'llama-7b',
              'temp': 0.7,
              'alpha': 0.95,
              'beta': 0.3,
              'basic': False,
              'settings': SerializerSettings(base=10, prec=3, signed=True, fixed_length=False, max_val=10000000.0, time_sep=' ,', bit_sep=' ', plus_sign='', minus_sign=' -', half_bin_correction=True, decimal_point='', missing_str=' Nan')})]

In [15]:
if isinstance(hypers,dict):
    hypers = list(grid_iter(hypers))
else:
    assert isinstance(hypers, list), 'hypers must be a list or dict'
if not isinstance(train, list):
    train = [train]
    test = [test]
n_val = len(train)

In [17]:
test

[date
 1989-03-18    353.5
 1989-03-25    354.4
 1989-04-01    354.5
 1989-04-08    355.0
 1989-04-15    355.4
               ...  
 2001-12-01    370.3
 2001-12-08    370.8
 2001-12-15    371.2
 2001-12-22    371.3
 2001-12-29    371.5
 Name: co2, Length: 668, dtype: float64]

In [18]:
best_hyper = hypers[0]
best_val_nll = float('inf')

In [19]:
best_hyper

defaultdict(dict,
            {'model': 'llama-7b',
             'temp': 0.7,
             'alpha': 0.95,
             'beta': 0.3,
             'basic': False,
             'settings': SerializerSettings(base=10, prec=3, signed=True, fixed_length=False, max_val=10000000.0, time_sep=' ,', bit_sep=' ', plus_sign='', minus_sign=' -', half_bin_correction=True, decimal_point='', missing_str=' Nan')})

In [23]:
for i in range(len(train)):
    if not isinstance(train[i], pd.Series):
        train[i] = pd.Series(train[i], index=pd.RangeIndex(len(train[i])))
        test[i] = pd.Series(test[i], index=pd.RangeIndex(len(train[i]), len(test[i])+len(train[i])))

In [26]:
train

[date
 1958-03-29    316.1
 1958-04-05    317.3
 1958-04-12    317.6
 1958-04-19    317.5
 1958-04-26    316.4
               ...  
 1989-02-11    352.6
 1989-02-18    353.2
 1989-02-25    353.4
 1989-03-04    353.1
 1989-03-11    353.4
 Name: co2, Length: 1557, dtype: float64]

In [27]:
test

[date
 1989-03-18    353.5
 1989-03-25    354.4
 1989-04-01    354.5
 1989-04-08    355.0
 1989-04-15    355.4
               ...  
 2001-12-01    370.3
 2001-12-08    370.8
 2001-12-15    371.2
 2001-12-22    371.3
 2001-12-29    371.5
 Name: co2, Length: 668, dtype: float64]

In [34]:
@dataclass
class Scaler:
    """
    Represents a data scaler with transformation and inverse transformation functions.

    Attributes:
        transform (callable): Function to apply transformation.
        inv_transform (callable): Function to apply inverse transformation.
    """
    transform: callable = lambda x: x
    inv_transform: callable = lambda x: x

def get_scaler(history, alpha=0.95, beta=0.3, basic=False):
    """
    Generate a Scaler object based on given history data.

    Args:
        history (array-like): Data to derive scaling from.
        alpha (float, optional): Quantile for scaling. Defaults to .95.
        # Truncate inputs
        tokens = [tokeniz]
        beta (float, optional): Shift parameter. Defaults to .3.
        basic (bool, optional): If True, no shift is applied, and scaling by values below 0.01 is avoided. Defaults to False.

    Returns:
        Scaler: Configured scaler object.
    """
    history = history[~np.isnan(history)]
    if basic:
        q = np.maximum(np.quantile(np.abs(history), alpha),.01)
        def transform(x):
            return x / q
        def inv_transform(x):
            return x * q
    else:
        min_ = np.min(history) - beta*(np.max(history)-np.min(history))
        q = np.quantile(history-min_, alpha)
        if q == 0:
            q = 1
        def transform(x):
            return (x - min_) / q
        def inv_transform(x):
            return x * q + min_
    return Scaler(transform=transform, inv_transform=inv_transform)

In [35]:
alpha=0.95
beta=0.3
basic=False
scalers = [get_scaler(train[i].values, alpha=alpha, beta=beta, basic=basic) for i in range(len(train))]

In [38]:
scalers[0]

Scaler(transform=<function get_scaler.<locals>.transform at 0x7fddb0cb6a60>, inv_transform=<function get_scaler.<locals>.inv_transform at 0x7fddb0cb1c10>)

In [42]:
input_arrs = [train[i].values for i in range(len(train))]
input_arrs

[array([316.1, 317.3, 317.6, ..., 353.4, 353.1, 353.4])]

In [43]:
transformed_input_arrs = np.array([scaler.transform(input_array) for input_array, scaler in zip(input_arrs, scalers)])

In [44]:
transformed_input_arrs

array([[0.31754135, 0.34204615, 0.34817235, ..., 1.07923218, 1.07310598,
        1.07923218]])

In [53]:
transformed_input_arrs[0][-1]

1.0792321829691647

In [45]:
from functools import partial

def vec_num2repr(val, base, prec, max_val):
    """
    Convert numbers to a representation in a specified base with precision.

    Parameters:
    - val (np.array): The numbers to represent.
    - base (int): The base of the representation.
    - prec (int): The precision after the 'decimal' point in the base representation.
    - max_val (float): The maximum absolute value of the number.

    Returns:
    - tuple: Sign and digits in the specified base representation.
    
    Examples:
        With base=10, prec=2:
            0.5   ->    50
            3.52  ->   352
            12.5  ->  1250
    """
    base = float(base)
    bs = val.shape[0]
    sign = 1 * (val >= 0) - 1 * (val < 0)
    val = np.abs(val)
    max_bit_pos = int(np.ceil(np.log(max_val) / np.log(base)).item())

    before_decimals = []
    for i in range(max_bit_pos):
        digit = (val / base**(max_bit_pos - i - 1)).astype(int)
        before_decimals.append(digit)
        val -= digit * base**(max_bit_pos - i - 1)

    before_decimals = np.stack(before_decimals, axis=-1)

    if prec > 0:
        after_decimals = []
        for i in range(prec):
            digit = (val / base**(-i - 1)).astype(int)
            after_decimals.append(digit)
            val -= digit * base**(-i - 1)

        after_decimals = np.stack(after_decimals, axis=-1)
        digits = np.concatenate([before_decimals, after_decimals], axis=-1)
    else:
        digits = before_decimals
    return sign, digits

def vec_repr2num(sign, digits, base, prec, half_bin_correction=True):
    """
    Convert a string representation in a specified base back to numbers.

    Parameters:
    - sign (np.array): The sign of the numbers.
    - digits (np.array): Digits of the numbers in the specified base.
    - base (int): The base of the representation.
    - prec (int): The precision after the 'decimal' point in the base representation.
    - half_bin_correction (bool): If True, adds 0.5 of the smallest bin size to the number.

    Returns:
    - np.array: Numbers corresponding to the given base representation.
    """
    base = float(base)
    bs, D = digits.shape
    digits_flipped = np.flip(digits, axis=-1)
    powers = -np.arange(-prec, -prec + D)
    val = np.sum(digits_flipped/base**powers, axis=-1)

    if half_bin_correction:
        val += 0.5/base**prec

    return sign * val

def serialize_arr(arr, settings: SerializerSettings):
    """
    Serialize an array of numbers (a time series) into a string based on the provided settings.

    Parameters:
    - arr (np.array): Array of numbers to serialize.
    - settings (SerializerSettings): Settings for serialization.

    Returns:
    - str: String representation of the array.
    """
    # max_val is only for fixing the number of bits in nunm2repr so it can be vmapped
    assert np.all(np.abs(arr[~np.isnan(arr)]) <= settings.max_val), f"abs(arr) must be <= max_val,\
         but abs(arr)={np.abs(arr)}, max_val={settings.max_val}"
    
    if not settings.signed:
        assert np.all(arr[~np.isnan(arr)] >= 0), f"unsigned arr must be >= 0"
        plus_sign = minus_sign = ''
    else:
        plus_sign = settings.plus_sign
        minus_sign = settings.minus_sign
    
    vnum2repr = partial(vec_num2repr,base=settings.base,prec=settings.prec,max_val=settings.max_val)
    sign_arr, digits_arr = vnum2repr(np.where(np.isnan(arr),np.zeros_like(arr),arr))
    ismissing = np.isnan(arr)
    
    def tokenize(arr):
        return ''.join([settings.bit_sep+str(b) for b in arr])
    
    bit_strs = []
    for sign, digits,missing in zip(sign_arr, digits_arr, ismissing):
        if not settings.fixed_length:
            # remove leading zeros
            nonzero_indices = np.where(digits != 0)[0]
            if len(nonzero_indices) == 0:
                digits = np.array([0])
            else:
                digits = digits[nonzero_indices[0]:]
            # add a decimal point
            prec = settings.prec
            if len(settings.decimal_point):
                digits = np.concatenate([digits[:-prec], np.array([settings.decimal_point]), digits[-prec:]])
        digits = tokenize(digits)
        sign_sep = plus_sign if sign==1 else minus_sign
        if missing:
            bit_strs.append(settings.missing_str)
        else:
            bit_strs.append(sign_sep + digits)
    bit_str = settings.time_sep.join(bit_strs)
    bit_str += settings.time_sep # otherwise there is ambiguity in number of digits in the last time step
    return bit_str

In [49]:
settings = best_hyper['settings']
if isinstance(settings, dict):
    settings = SerializerSettings(**settings)

In [50]:
input_strs = [serialize_arr(scaled_input_arr, settings) for scaled_input_arr in transformed_input_arrs]
input_strs

[' 3 1 7 , 3 4 2 , 3 4 8 , 3 4 6 , 3 2 3 , 3 3 3 , 3 4 6 , 3 5 4 , 3 1 1 , 3 1 1 , 3 0 3 , 3 0 5 , 3 0 7 , 2 9 7 , 2 9 5 , 2 7 6 , 2 6 4 , 2 5 4 , 2 5 8 , 2 6 4 , 2 7 4 , 2 8 4 , 2 8 2 , 2 8 8 , 2 9 9 , 2 9 9 , 3 0 5 , 3 0 7 , 3 1 1 , 3 0 3 , 3 3 3 , 3 2 7 , 3 2 7 , 3 3 1 , 3 2 9 , 3 2 9 , 3 5 0 , 3 3 7 , 3 4 8 , 3 6 2 , 3 6 0 , 3 7 0 , 3 5 6 , 3 6 4 , 3 6 6 , 3 5 8 , 3 5 2 , 3 5 0 , 3 3 1 , 3 3 1 , 3 2 3 , 3 1 7 , 3 0 7 , 2 9 3 , 2 9 5 , 2 7 6 , 2 8 2 , 2 7 2 , 2 6 4 , 2 6 4 , 2 5 4 , 2 5 6 , 2 6 2 , 2 6 2 , 2 7 6 , 2 8 2 , 2 9 0 , 2 9 9 , 2 9 7 , 2 9 5 , 3 0 7 , 3 1 1 , 3 0 9 , 3 0 9 , 3 2 3 , 3 2 9 , 3 2 5 , 3 2 7 , 3 2 7 , 3 3 3 , 3 4 4 , 3 3 5 , 3 3 3 , 3 5 0 , 3 5 6 , 3 5 0 , 3 6 8 , 3 8 2 , 3 7 6 , 3 7 6 , 3 9 1 , 3 9 5 , 3 9 3 , 3 9 7 , 3 9 7 , 3 8 4 , 3 9 7 , 3 8 4 , 3 7 6 , 3 5 8 , 3 6 8 , 3 6 4 , 3 5 4 , 3 4 2 , 3 2 7 , 3 2 9 , 2 9 7 , 2 8 8 , 2 8 8 , 2 8 4 , 2 7 8 , 2 6 0 , 2 6 6 , 2 6 0 , 2 7 2 , 2 7 8 , 2 7 8 , 2 8 4 , 2 9 7 , 2 9 7 , 3 0 3 , 3 1 1 , 3 1 5 , 3 1 9 , 3 2 3

In [54]:
DEFAULT_EOS_TOKEN = "</s>"
DEFAULT_BOS_TOKEN = "<s>"
DEFAULT_UNK_TOKEN = "<unk>"

from transformers import (
    LlamaForCausalLM, 
    LlamaTokenizer, 
)

def get_tokenizer(model):
    name_parts = model.split("-")
    model_size = name_parts[0]
    chat = len(name_parts) > 1
    assert model_size in ["7b", "13b", "70b"]
    
    chat = "chat-" if chat else ""

    tokenizer = LlamaTokenizer.from_pretrained(
        f"meta-llama/Llama-2-{model_size.lower()}-{chat}hf",
        use_fast=False,
    )

    special_tokens_dict = dict()
    if tokenizer.eos_token is None:
        special_tokens_dict["eos_token"] = DEFAULT_EOS_TOKEN
    if tokenizer.bos_token is None:
        special_tokens_dict["bos_token"] = DEFAULT_BOS_TOKEN
    if tokenizer.unk_token is None:
        special_tokens_dict["unk_token"] = DEFAULT_UNK_TOKEN

    tokenizer.add_special_tokens(special_tokens_dict)
    tokenizer.pad_token = tokenizer.eos_token

    return tokenizer

def tokenize_fn(str, model):
    tokenizer = get_tokenizer(model)
    return tokenizer(str)

def truncate_input(input_arr, input_str, settings, steps):
    """
    Truncate inputs to the maximum context length for a given model.
    """
    tokenization_fn = partial(tokenize_fn, model='7b')
    context_length = 4096
    input_str_chuncks = input_str.split(settings.time_sep)
    for i in range(len(input_str_chuncks) - 1):
        truncated_input_str = settings.time_sep.join(input_str_chuncks[i:])
        # add separator if not already present
        if not truncated_input_str.endswith(settings.time_sep):
            truncated_input_str += settings.time_sep
        input_tokens = tokenization_fn(truncated_input_str)
        num_input_tokens = len(input_tokens)
        avg_token_length = num_input_tokens / (len(input_str_chuncks) - i)
        STEP_MULTIPLIER = 1.2
        num_output_tokens = avg_token_length * steps * STEP_MULTIPLIER
        if num_input_tokens + num_output_tokens <= context_length:
            truncated_input_arr = input_arr[i:]
            break
    if i > 0:
        print(f'Warning: Truncated input from {len(input_arr)} to {len(truncated_input_arr)}')
    return truncated_input_arr, truncated_input_str

In [56]:
input_arrs, input_strs = zip(*[truncate_input(input_array, input_str, settings, len(test[0])) for input_array, input_str in zip(input_arrs, input_strs)])

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

In [57]:
input_arrs

(array([316.1, 317.3, 317.6, ..., 353.4, 353.1, 353.4]),)

In [58]:
input_strs

(' 3 1 7 , 3 4 2 , 3 4 8 , 3 4 6 , 3 2 3 , 3 3 3 , 3 4 6 , 3 5 4 , 3 1 1 , 3 1 1 , 3 0 3 , 3 0 5 , 3 0 7 , 2 9 7 , 2 9 5 , 2 7 6 , 2 6 4 , 2 5 4 , 2 5 8 , 2 6 4 , 2 7 4 , 2 8 4 , 2 8 2 , 2 8 8 , 2 9 9 , 2 9 9 , 3 0 5 , 3 0 7 , 3 1 1 , 3 0 3 , 3 3 3 , 3 2 7 , 3 2 7 , 3 3 1 , 3 2 9 , 3 2 9 , 3 5 0 , 3 3 7 , 3 4 8 , 3 6 2 , 3 6 0 , 3 7 0 , 3 5 6 , 3 6 4 , 3 6 6 , 3 5 8 , 3 5 2 , 3 5 0 , 3 3 1 , 3 3 1 , 3 2 3 , 3 1 7 , 3 0 7 , 2 9 3 , 2 9 5 , 2 7 6 , 2 8 2 , 2 7 2 , 2 6 4 , 2 6 4 , 2 5 4 , 2 5 6 , 2 6 2 , 2 6 2 , 2 7 6 , 2 8 2 , 2 9 0 , 2 9 9 , 2 9 7 , 2 9 5 , 3 0 7 , 3 1 1 , 3 0 9 , 3 0 9 , 3 2 3 , 3 2 9 , 3 2 5 , 3 2 7 , 3 2 7 , 3 3 3 , 3 4 4 , 3 3 5 , 3 3 3 , 3 5 0 , 3 5 6 , 3 5 0 , 3 6 8 , 3 8 2 , 3 7 6 , 3 7 6 , 3 9 1 , 3 9 5 , 3 9 3 , 3 9 7 , 3 9 7 , 3 8 4 , 3 9 7 , 3 8 4 , 3 7 6 , 3 5 8 , 3 6 8 , 3 6 4 , 3 5 4 , 3 4 2 , 3 2 7 , 3 2 9 , 2 9 7 , 2 8 8 , 2 8 8 , 2 8 4 , 2 7 8 , 2 6 0 , 2 6 6 , 2 6 0 , 2 7 2 , 2 7 8 , 2 7 8 , 2 8 4 , 2 9 7 , 2 9 7 , 3 0 3 , 3 1 1 , 3 1 5 , 3 1 9 , 3 2 3

In [59]:
steps = len(test[0])
samples = None
medians = None
completions_list = None